In [14]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sequential import Sequential
from conv2d import Conv2D
from pooling2d import Pooling2D
from flatten import Flatten
from dense import Dense

In [15]:
np.random.seed(13517013)

In [16]:
image_size = (30, 30)

In [17]:
def readImage(path):
    result = []
    images = os.listdir(path)
    print(images)
    for image in images:
        result.append(cv2.resize(cv2.imread(path + '/' + image, 1), image_size))
    return np.array(result)

In [18]:
cats = readImage("./cats")

['cat.0.jpg', 'cat.15.jpg', 'cat.17.jpg', 'cat.19.jpg', 'cat.2.jpg', 'cat.21.jpg', 'cat.23.jpg', 'cat.26.jpg', 'cat.36.jpg', 'cat.38.jpg', 'cat.40.jpg', 'cat.45.jpg', 'cat.48.jpg', 'cat.49.jpg', 'cat.50.jpg', 'cat.58.jpg', 'cat.60.jpg', 'cat.61.jpg', 'cat.71.jpg', 'cat.9.jpg']


In [19]:
dogs = readImage("./dogs")

['dog.0.jpg', 'dog.24.jpg', 'dog.25.jpg', 'dog.3.jpg', 'dog.30.jpg', 'dog.34.jpg', 'dog.36.jpg', 'dog.38.jpg', 'dog.4.jpg', 'dog.40.jpg', 'dog.42.jpg', 'dog.5.jpg', 'dog.50.jpg', 'dog.51.jpg', 'dog.58.jpg', 'dog.60.jpg', 'dog.61.jpg', 'dog.63.jpg', 'dog.8.jpg', 'dog.9.jpg']


In [20]:
xdata = np.concatenate((cats, dogs))
yData = np.array([0] * len(cats) + [1] * len(dogs))

x_train, x_test, y_train, y_test = train_test_split(
        xdata, yData, test_size=0.1, random_state=13517013)

print(xdata.shape)

(40, 30, 30, 3)


In [21]:
model = Sequential()

model.add(Conv2D(8, (2, 2), pad=0, stride=2, input_shape=(xdata.shape[1], xdata.shape[2], xdata.shape[3]), activation='relu'))
model.add(Pooling2D((2, 2), stride=2, pool_mode='avg'))
model.add(Flatten())
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(4, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [22]:
pred = model.forward(x_train)
print(pred)

[[0.5507473 ]
 [0.54416562]
 [0.54416481]
 [0.5446854 ]
 [0.75907003]
 [0.54416481]
 [0.54416476]
 [0.54416787]
 [0.54416481]
 [0.5459428 ]
 [0.54417641]
 [0.54416573]
 [0.54416481]
 [0.54416495]
 [0.55333302]
 [0.5442836 ]
 [0.54416482]
 [0.54433296]
 [0.54416555]
 [0.54416481]
 [0.54588208]
 [0.54416481]
 [0.54416493]
 [0.54416481]
 [0.54420093]
 [0.54425899]
 [0.54416481]
 [0.54416482]
 [0.54416509]
 [0.54811108]
 [0.54429292]
 [0.54416486]
 [0.54422253]
 [0.54416582]
 [0.54416499]
 [0.54418194]]


In [23]:
print(classification_report(y_train, np.round(pred)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.50      1.00      0.67        18

    accuracy                           0.50        36
   macro avg       0.25      0.50      0.33        36
weighted avg       0.25      0.50      0.33        36



c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [24]:
print('Initial error:', np.mean(model.calculateError(y_train, pred)))

Initial error: 0.6882761030892843


In [25]:
# Make labels from possibility values
def makeLabels(pred):
    result = []
    for each_pred in pred:
        if each_pred >= 0.5:
            result.append("dogs")
        else:
            result.append("cats")
    return np.array(result)

print("Predicted Array")
print(makeLabels(pred))
print("Actual Array")
print(makeLabels(y_train))

Predicted Array
['dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs'
 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs'
 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs'
 'dogs' 'dogs' 'dogs' 'dogs' 'dogs' 'dogs']
Actual Array
['dogs' 'dogs' 'cats' 'dogs' 'dogs' 'dogs' 'dogs' 'cats' 'cats' 'dogs'
 'cats' 'cats' 'cats' 'cats' 'cats' 'dogs' 'dogs' 'cats' 'dogs' 'dogs'
 'cats' 'cats' 'cats' 'dogs' 'dogs' 'cats' 'cats' 'cats' 'cats' 'cats'
 'dogs' 'dogs' 'dogs' 'dogs' 'cats' 'dogs']


In [26]:
# Using the backprop method to train the model
history = model.fit(x_train, y_train, epochs=5, lr=0.04, momentum=0.3, lr_decay=0, batch_size=10)

Learning rate: 0.04
Update weight
<conv2d.Conv2D object at 0x0000020D3DA6B880> [[[[ 1.52717890e-06  1.52717890e-06  1.52717890e-06]
   [-5.94654447e-06 -5.94654447e-06 -5.94654447e-06]]

  [[ 1.15437222e-06  1.15437222e-06  1.15437222e-06]
   [ 3.76076438e-06  3.76076438e-06  3.76076438e-06]]]] [0.]
<pooling2d.Pooling2D object at 0x0000020D3DA6B940> [0.] [0.]
<flatten.Flatten object at 0x0000020D3DA6B9A0> [0.] [0.]
<dense.Dense object at 0x0000020D3DA6B970> [[-1.70562416e-04 -9.31236916e-31 -2.80097151e-84  9.35114177e-04
   0.00000000e+00  1.72247253e-47 -5.74239936e-10  1.22014956e-11]] [[-1.14308834e-07 -7.47195381e-34 -3.48677747e-87  1.07516968e-06
   0.00000000e+00  2.65419978e-50 -5.42582335e-13  1.00520973e-14]]
<dense.Dense object at 0x0000020D7A508910> [[ 0.00000000e+00 -5.43757823e-06 -3.33590998e-10 -1.19237460e-13]] [[ 0.00000000e+00  7.46003096e-05 -2.08494374e-09 -1.49047104e-12]]
<dense.Dense object at 0x0000020D3C74F7C0> [[0.0056928]] [[0.04572976]]
-------------


c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.50      1.00      0.67        18

    accuracy                           0.50        36
   macro avg       0.25      0.50      0.33        36
weighted avg       0.25      0.50      0.33        36

Epoch: 1 = error :  0.6852357083306732


Learning rate: 0.04
Update weight
<conv2d.Conv2D object at 0x0000020D3DA6B880> [[[[ 3.27409224e-07  3.27409224e-07  3.27409224e-07]
   [-1.43071578e-06 -1.43071578e-06 -1.43071578e-06]]

  [[-6.79012287e-07 -6.79012287e-07 -6.79012287e-07]
   [ 1.33387043e-06  1.33387043e-06  1.33387043e-06]]]] [0.]
<pooling2d.Pooling2D object at 0x0000020D3DA6B940> [0.] [0.]
<flatten.Flatten object at 0x0000020D3DA6B9A0> [0.] [0.]
<dense.Dense object at 0x0000020D3DA6B970> [[-3.60161087e-04 -9.56196326e-31 -2.28134677e-84  2.80764202e-04
   0.00000000e+00  1.40527339e-47 -6.11409103e-10  1.27588401e-11]] [[-3.60683075e-07 -7.67222414e-34 -2

In [27]:
pred_test = model.forward(x_test)
print(classification_report(y_test, np.round(pred_test)))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       0.00      0.00      0.00         2

    accuracy                           0.50         4
   macro avg       0.25      0.50      0.33         4
weighted avg       0.25      0.50      0.33         4



c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alexa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo